<a href="https://colab.research.google.com/github/prof-Thiago-Nunes/Computer_vision/blob/main/YOLOv9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [9]:
# Clona o repositório do YOLOv9
!git clone https://github.com/prof-Thiago-Nunes/yolov9.git

# Muda o diretório de trabalho atual para o repositório YOLOv9 clonado
%cd yolov9

# Instala as dependências necessárias do YOLOv9 a partir do arquivo requirements.txt
!pip install -r requirements.txt -q

Cloning into 'yolov9'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 164 (delta 26), reused 16 (delta 16), pack-reused 127
Receiving objects: 100% (164/164), 619.17 KiB | 13.17 MiB/s, done.
Resolving deltas: 100% (66/66), done.
/content/yolov9/yolov9


In [10]:
# Importa bibliotecas necessárias
import sys
import requests
from tqdm.notebook import tqdm
from pathlib import Path
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams


# Configuração de diretórios para código e dados
CODE_FOLDER = Path("..").resolve()  # Diretório do código
WEIGHTS_FOLDER = CODE_FOLDER / "weights"  # Diretório para pesos do modelo
DATA_FOLDER = CODE_FOLDER / "data"  # Diretório para dados

# Cria os diretórios para pesos e dados, se não existirem
WEIGHTS_FOLDER.mkdir(exist_ok=True, parents=True)
DATA_FOLDER.mkdir(exist_ok=True, parents=True)

# Adiciona o diretório do código ao path do Python para importação de módulos
sys.path.append(str(CODE_FOLDER))

rcParams['figure.figsize'] = 15, 15
%matplotlib inline

In [11]:
# URLs dos arquivos de pesos
weight_files = [
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt",
    "https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt"
]

# Itera sobre a lista de URLs para baixar os arquivos de pesos
for i, url in enumerate(weight_files, start=1):
    filename = url.split('/')[-1]
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, desc=f"Baixando arquivo {i}/{len(weight_files)}: {filename}")
    with open(WEIGHTS_FOLDER / filename, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

Baixando arquivo 1/4: yolov9-c.pt:   0%|          | 0.00/103M [00:00<?, ?iB/s]

Baixando arquivo 2/4: yolov9-e.pt:   0%|          | 0.00/140M [00:00<?, ?iB/s]

Baixando arquivo 3/4: gelan-c.pt:   0%|          | 0.00/51.5M [00:00<?, ?iB/s]

Baixando arquivo 4/4: gelan-e.pt:   0%|          | 0.00/117M [00:00<?, ?iB/s]

In [12]:
# URL da imagem de teste
url = 'https://sigmoidal.ai/wp-content/uploads/2022/11/314928609_1293071608150779_8666358890956473002_n.jpg'

# Faz a requisição para obter a imagem
response = requests.get(url)

# Define o caminho do arquivo onde a imagem será salva dentro do DATA_FOLDER
image_path = DATA_FOLDER / "test_image.jpg"

# Salva a imagem no diretório especificado
with open(image_path, 'wb') as f:
    f.write(response.content)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
!python {CODE_FOLDER}/detect.py --weights {WEIGHTS_FOLDER}/yolov9-e.pt --conf 0.1 --source {DATA_FOLDER}/test_image.jpg --device cpu

detect: weights=['/content/yolov9/weights/yolov9-e.pt'], source=/content/yolov9/data/test_image.jpg, data=../data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 26e3791 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 1119 layers, 69470144 parameters, 0 gradients, 244.0 GFLOPs
image 1/1 /content/yolov9/data/test_image.jpg: 640x608 6 persons, 1 sports ball, 17 bottles, 4 cups, 2 potted plants, 1 dining table, 3 books, 5854.7ms
Speed: 4.4ms pre-process, 5854.7ms inference, 32.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to ../runs/detect/exp


In [14]:
!python {CODE_FOLDER}/detect.py --weights {WEIGHTS_FOLDER}/yolov9-e.pt --conf 0.1 --source {DATA_FOLDER}/rua_1.mp4 --device cpu

detect: weights=['/content/yolov9/weights/yolov9-e.pt'], source=/content/yolov9/data/rua_1.mp4, data=../data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 26e3791 Python-3.10.12 torch-2.1.0+cu121 CPU

Fusing layers... 
Model summary: 1119 layers, 69470144 parameters, 0 gradients, 244.0 GFLOPs
Traceback (most recent call last):
  File "/content/yolov9/detect.py", line 233, in <module>
    main(opt)
  File "/content/yolov9/detect.py", line 228, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolov9/detect.